In [50]:
import pandas as pd
import numpy as np
import Actu_Colums as Ac
import win32com.client as win32
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.worksheet.table import Table, TableStyleInfo
import datetime
import os
import glob
import pyxlsb
import tqdm
from typing import Optional
from collections import defaultdict
from babel.dates import format_date, format_datetime, Locale
import pickle
import calendar
import locale
import seaborn as sns

### El flujo de actualizacion es el siguiente : 
    -01.Tomo toda el reporte de PREPA
    -02.Actualizo info del PAP(Seudo-Merge)
    -03.Fill DATA y agrego antiguacion a los sin registrar
    -04.Mapeo para crear las nuevas etapas
    -05.To CSV (Capa plata)
    -06.Agrupo y Comparo para crear medida de diferencia (Capa Gold)

# Funciones 

In [51]:
Today_str = datetime.date.today().strftime('%d-%m-%Y')
Today_D_M = Today_str[0:5]
Today_D_M = [Today_D_M[0:2],'.',Today_D_M[3:6]]
Today_D_M = ''.join(Today_D_M)
#Today_D_M = '06.02'
print('Hoy es:',Today_str)

def transformar_name(valor):
    valor = str(valor)
    if valor.isupper():
        return valor
    else:
        try:
            palabras = valor.split()
            frase = ' '.join([palabras[0], palabras[2]])
            return frase
        except:
            return valor
        
def process_df(df: pd.DataFrame,clases_unicas: list,index=['NOMPROVEEDOR']):
    """ La funcion recibe un df, y las clases unicas
        Y devuelve el df uniformizado para cada clase
        Llena los valores faltantes con 0"""
    pivot_table = pd.pivot_table(df, values='PEND_FACT_SOLES', index=index+['ESTADO'], aggfunc=pd.Series.sum) #Agrupo 
    grupos = pivot_table.groupby(index) #creo un data frame para cada contrata
    # Para cada grupo, crea un nuevo dataframe y guárdalo en un diccionario
    dataframes = {}
    for nombre, datos in grupos: # Creo los df dentro del dic
        dataframes[nombre] = datos
    for key in dataframes.keys(): # para cada df en el dic
        df = dataframes[key].reset_index()
        ##display(df)
        #display(clases_unicas)
        df1 = df.set_index('ESTADO').reindex(clases_unicas).reset_index() #normalizo, creando filas para todos los ESTADOS
        df1.RESPONSABLE2 = key[0]
        df1.NOMPROVEEDOR = key[1]  #Relleno los NAN  #Relleno los NAN
        df1 = df1.fillna(0)  
        dataframes[key] = df1   #Reescribo los Df para cada clave
    df_concat = pd.concat(dataframes.values(),ignore_index=True) # Compacto todo los df del dic en uno grande 
    pivot_table_1 = pd.pivot_table(df_concat, values='PEND_FACT_SOLES', index= index+['ESTADO'] , aggfunc=pd.Series.sum) #agrupo de nuevo
    return pivot_table_1

def calc_diff(df1: pd.DataFrame,df2: pd.DataFrame, index= ['NOMPROVEEDOR', 'ESTADO']) -> pd.DataFrame:
    """ entran 2 dataframes , los proceso , y en base a los df procesados 
        calculo la diferencia en una nueva columna, 
        y añado esta columna de diferencia al df actual,
        DF1 - DF2"""
    clases_unicas = pd.concat([df1['ESTADO'], df2['ESTADO']]).unique() #Creo listas de claves unicas
    df1_proces = process_df(df1,clases_unicas,index)
    df2_proces = process_df(df2,clases_unicas,index)
    
    df_diff = df1_proces - df2_proces
    diff_pivot_table_reset = df_diff.reset_index()
    diff_pivot_table_reset.rename(columns={'PEND_FACT_SOLES': 'DIFERENCIA EN SOLES'}, inplace=True)
    # Merge diff_pivot_table_filled_reset con PrePa_O_EI
    PrePa_O_EI = pd.merge(df1, diff_pivot_table_reset, on=index+['ESTADO'],how='left')    
    return PrePa_O_EI
    
def new_line_TS(df : pd.DataFrame,ruta_del_csv : str,Fecha: str,index:list): 
    df['TIME'] = Fecha
    pivot_table = pd.pivot_table(df, values='PEND_FACT_SOLES', index=index, aggfunc=pd.Series.sum) #Agrupo 
    df_reset = pivot_table.reset_index(drop=False)
    #convetir la columna del agrupado al formato d efehca 
    # Convierte la columna 'Fecha' a datetime
    df_reset['TIME'] = pd.to_datetime(df_reset['TIME'],format = '%d-%m-%Y')
    
    # Formatea la columna 'Fecha'
    df_reset['TIME Format'] = df_reset['TIME'].apply(lambda x: format_date(x, 'EEE dd-MM-yyyy', locale=Locale('es', 'ES')))
    df_reset.to_csv(ruta_del_csv, mode='a', header=False,index=False)
    

# Definimos los valores que estamos buscando en una lista
def map_estados(PRE_all_act:pd.DataFrame):
    PRE_all_act.loc[PRE_all_act['ESTADO_PAP'].isna(), 'ESTADO_PAP'] = 'Sin Registrar' #Reemplazo valor para determinada condicion 
    
    PRE_all_act['RESPONSABLE_PAP'] = PRE_all_act['RESPONSABLE_PAP'].replace('nan','SIN RESPONSABLE')
    PRE_all_act['RESPONSABLE_PAP'] = PRE_all_act['RESPONSABLE_PAP'].replace('','SIN RESPONSABLE')
    PRE_all_act['RESPONSABLE_PAP'] = PRE_all_act['RESPONSABLE_PAP'].replace('Luis Romero','Luis - Romero')## valor atipico
    
    # Aplicar la transformación
    PRE_all_act['RESPONSABLE_PAP'] = PRE_all_act['RESPONSABLE_PAP'].apply(transformar_name)    
    valores_buscados = ['Aprobaciones FAC', 'Aprobaciones PAC']
    
    # Creamos la nueva columna basada en la condición
    PRE_all_act['ESTADO'] = np.where(PRE_all_act['ESTADO_PAP'].isin(valores_buscados), 
                                     PRE_all_act['RESPONSABLE_PAP'], 
                                     PRE_all_act['ESTADO_PAP'])
 
    map_estado = {'Auto ATP' : 'Registrar lista de pendientes',
           'Visita ejecutada' : 'Registrar lista de pendientes',
            'Programado' : 'En ATP',
            'FAC' : 'Solic FAC',
            'PAC': 'Solic PAC',
            'Programación ATP':'Programar ATP',
            'Registrado':'Programar ATP','Lista Pendientes':'Validar PL.'}
    PRE_all_act.loc[PRE_all_act['ESTADO'] == 'SIN RESPONSABLE', 'ESTADO'] = 'Por Asignar'
    PRE_all_act['ESTADO'] = PRE_all_act['ESTADO'].replace(map_estado)
    return PRE_all_act
    
def update_from_PAP(Prepa_PR:pd.DataFrame,PAP_S:pd.DataFrame):
    unhavent_columns = set(PAP_S.columns.tolist()) - set(Prepa_PR.columns.tolist()) # Columnas que le faltan al grande del chico
    # Crear un nuevo DataFrame con las columnas necesarias
    new_columns = pd.DataFrame(index=Prepa_PR.index, columns=list(unhavent_columns), dtype='object')
    
    # Concatenar el nuevo DataFrame con el original
    Prepa_PR = pd.concat([Prepa_PR, new_columns], axis=1)
    PRE_all_act =  Ac.update_values_optimized_V2(Prepa_PR, PAP_S, "OC Posición",['PAP','SITE','ID Site','ESTADO_PAP','F.Creación',
                                                'RESPONSABLE_PAP','ANTIGUAMIENTO_PAP','Gerencia','F.Modifica']) 
    duplicated_index = PRE_all_act.index.duplicated(keep=False)
    df_duplicated = PRE_all_act[duplicated_index].copy()
    PRE_all_act.drop(PRE_all_act[duplicated_index].index, inplace=True) ## dropeo las duplicadas
    # Ordena el DataFrame por la columna de fechas  LA FECHA DE CREACION ME SIRVE PARA ELMINAR DUPLICADOS
    df_duplicated['F.Modifica'] = pd.to_datetime(df_duplicated['F.Modifica'], format="%d/%m/%Y %I:%M:%S %p") # Paso a fecha para ordenar
    df_duplicated.sort_values('F.Modifica', inplace=True)
    df_duplicated_NR = df_duplicated[df_duplicated.ESTADO_PAP != 'Rechazado'].sort_index(ascending=False)
    df_duplicated_NRF = df_duplicated_NR.loc[~df_duplicated_NR.index.duplicated(keep='first')]
    #df_duplicated['F.Creación'] = df_duplicated['F.Creación'].dt.strftime("%m/%d/%Y %I:%M:%S %p") # Regreso a str 
    PRE_all_act = pd.concat([PRE_all_act, df_duplicated_NRF])
    Cash_out = PRE_all_act.PEND_FACT_SOLES.sum()
    PRE_all_act.RESPONSABLE_PAP = PRE_all_act.RESPONSABLE_PAP.astype(str)
    return PRE_all_act

def load_df_by_name(directorios:str, cadena:str) -> pd.DataFrame:
    df_list = []
    for dictory in directorios:
        ruta = Ac.load_df_by_name(dictory, cadena)
        try:
            df = pd.read_excel(ruta, sheet_name='Hoja2')
            #print('Machea con nombre de hoja2')
        except Exception as e:
            print(f"Error: {e}. Trying with 'Sheet1'")
            df = pd.read_excel(ruta, sheet_name='Sheet1')
            #print('Machea con nombre de Sheet1')
        print(ruta)
        df_list.append(df)
    return df_list

def Merge_data(PAP:pd.DataFrame,Prepa:pd.DataFrame):
    """Toma un segmento del PAP,lo procesa. Procesa la base y hace un merge con el PAP,
        Mapea los estados, dropea las filas de PENDIENTE en caso exista la columna STATUSFINAL"""
####################Preproces PAP################################################
    PAP_f = PAP[['OC Posición','N° Sol','Estado','Nombre responsable','Id.SIte',
                 'SIte','# Días','F.Creación','Gerencia','F.Modifica']] ## Selecciona un Segmento de columnas del PAP 

    PAP_f = PAP_f.dropna(subset=['Id.SIte'])
    PAP_f = PAP_f[PAP_f['# Días'] < 500]
    
    #PAP_f_N = PAP_f[PAP_f['Id.SIte'].str.startswith(('L','T','SAD','CL','CAC'))].copy()
    PAP_f_N = PAP_f.rename(columns={'N° Sol': 'PAP', #Rename
                                   'SIte': 'SITE',
                                   'Estado': 'ESTADO_PAP',
                                    '# Días': 'ANTIGUAMIENTO_PAP',
                                    'Id.SIte': 'ID Site',
                                 'Nombre responsable': 'RESPONSABLE_PAP' })
    PAP_S = Ac.split_ocs(PAP_f_N) # Spliteo OCs    
    PAP_S['OC Posición'] = PAP_S['OC Posición'].astype(str)
###################Pre proces Prepa####################################################
    Prepa_CI = Prepa[Prepa.SUB_DIRECCION == 'CONS E IMP']
    Prepa_PR = Ac.convert_columns_to_str(Prepa_CI.copy(),['DOC_COMPRAS', 'POSIC','CONCA'])
    Prepa_PR.loc[:, "OC Posición"] = Prepa_PR.loc[:, "DOC_COMPRAS"].str.cat(Prepa_PR.loc[:, "POSIC"], sep= ":")
    Monto_In = Prepa_PR["PEND_FACT_SOLES"].sum() #Mont of USD 
    Prepa_PR = Prepa_PR.dropna(axis=1,how='all')
########################### MERGE DATAFRAMES ###############################################
    PRE_all_act = update_from_PAP(Prepa_PR,PAP_S)
    Cash_out = PRE_all_act['PEND_FACT_SOLES'].sum()
    print(Monto_In - Cash_out)    
    PRE_all_act= map_estados(PRE_all_act)
    try:
        PRE_all_act = PRE_all_act[PRE_all_act["STATUSFINAL"] == 'PENDIENTE']
        print("Se encuentra la columan 'STATUSFINAL' y se dropeand las filas de mas")
    except:
        pass        
    return PRE_all_act    


def find_ocs_sinSITE(df:pd.DataFrame):
    list_of_OCs = df[df.SITE.isna()]['DOC_COMPRAS'].unique().tolist()
    if not list_of_OCs:
        print('No hay Filas sin SITE')
    else:
        resultado = [valor[4:] for valor in list_of_OCs] # Tomo los valores luego del 4500
        a,b= Ac.MostCL_prefix(resultado)
        print("Faltan: ",len(resultado)," OCS")
        display(b)

contrata_dic = {'DELTA ELECTRONICS (PERU) INC. S.R.L ELTEK PERU S.R.L.':'DELTA',
                'COMUNICACION FUTURA SOCIEDAD ANONIM':'COMFUTURA'}
columns_dic = {"DOC COMPRAS": "Orden de Compra" ,
                "NOMPROVEEDOR" :"CONTRATISTA"
               , "PAP":"N° PAP", "PEND FACT SOLES":"PENDIENTE EN SOLES"
               , "TEXTO BREVE": "DESCRIPCIÓN"
               , "POSIC":"POSICIÓN"
               , "ANTIGUAMIENTO PAP" : "ANTIGUAMIENTO PAP"
               , "MES   COMPROMISO" : "MES DE COMPROMISO"
               , 'FECH CONTAB':'FECHA CONTABILIDAD'}

def formated2bi(df_diff:pd.DataFrame,contrata_dic:dict=contrata_dic,columns_dic:dict=columns_dic):
    def transformar_nombre(nombre):
          nombre = nombre.upper()
          nombre = nombre.replace('_', ' ')
          nombre = nombre.replace('-', ' ')
          return nombre

    df_fomarted = df_diff.copy()
    # Aplicar la función a los nombres de las columnas
    df_fomarted.columns = map(transformar_nombre, df_fomarted.columns) # Normalizo nombres de columna
    df_fomarted['SITE'] = df_fomarted['SITE'].str.replace('_', ' ')
    df_fomarted['SITE'] = df_fomarted['SITE'].str.title()
    df_fomarted['TEXTO BREVE'] = df_fomarted['TEXTO BREVE'].str.title()
    #df_fomarted['NOMPROVEEDOR'].replace(contrata_dic,inplace=True)
    df_fomarted['NOMPROVEEDOR'] = df_fomarted['NOMPROVEEDOR'].str.title()
    
    #Cambio los nombres de las columnas con un Map
    df_fomarted.rename(columns=columns_dic, inplace=True)
    #display(df_fomarted.info())
    columns_2_drop = ["EA EM","PEND FACT", "CONCATENADO",'TIPO MAT'
                          , "ELEMENTO PEP","FONDO","INDICADOR", "RESPONSABLE", "ANT PROYECTADO","SUB DIRECCION","DÍAS  2"]
    df_fomarted.drop(columns=columns_2_drop,inplace=True)
    df_fomarted.columns = df_fomarted.columns.str.title()
    return df_fomarted
    
def load_df_by_name(directorios:str, cadena:str) -> pd.DataFrame:
    df_list = []
    for dictory in directorios:
        ruta = Ac.buscar_archivo_mas_antiguo(dictory, cadena)
        try:
            df = pd.read_excel(ruta, sheet_name='Hoja2')
            #print('Machea con nombre de hoja2')
        except Exception as e:
            print(f"Error: {e}. Trying with 'Sheet1'")
            df = pd.read_excel(ruta, sheet_name='Sheet1')
            #print('Machea con nombre de Sheet1')
        #print(ruta)
        df_list.append(df)
    return df_list
def get_recent_df(Carpeta_path: str,sheet_name='Hoja2',extension='.xlsx'):
    """ Devuelve el df de la hoja especifica, del archivo mas reciente de la carpeta especificada"""
    Path_n= Carpeta_path + '\*'
    tipo_de_archivo = f'*{extension}'
    # Busca el archivo más reciente
    archivos = glob.glob(Path_n + tipo_de_archivo)
    archivo_mas_reciente = max(archivos, key=os.path.getctime)
    nombre_del_archivo_N = os.path.basename(archivo_mas_reciente)
    print(archivo_mas_reciente)
    if extension == '.xlsx':
        df = pd.read_excel(archivo_mas_reciente , sheet_name=sheet_name) 
    if extension == '.csv':
        df = pd.read_csv(archivo_mas_reciente)
    return df
def cleanrows(df):
    if 'STATUSFINAL' in df.columns:
        df = df[df['STATUSFINAL'] == 'PENDIENTE']
    elif 'STATUS FINAL' in df.columns:
        df = df[df['STATUS FINAL'] == 'PENDIENTE']
    return df

Hoy es: 30-07-2024


## Cargo Prepa

In [52]:
Path_n= 'D:/Scripts1/Code/ActPEA/archvis/Pre_pa/NEW/*'
tipo_de_archivo = '*.xlsx'

# Busca el archivo más reciente
#archivos
archivos = glob.glob(Path_n + tipo_de_archivo)
archivo_mas_reciente = max(archivos, key=os.path.getctime)
nombre_del_archivo_N = os.path.basename(archivo_mas_reciente)
nombre_del_archivo_N
Prepa_N = pd.read_excel(Path_n[0:-1] + nombre_del_archivo_N, sheet_name='DATA') 
print(archivo_mas_reciente)
Prepa_N = cleanrows(Prepa_N)
#aarchivos = glob.glob(Path_n + tipo_de_archivo)

D:/Scripts1/Code/ActPEA/archvis/Pre_pa/NEW\7. Prepasivo completo (Finanzas Julio).xlsx


In [53]:
Prepa_N.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1242 entries, 1 to 1847
Data columns (total 52 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   DOC_COMPRAS           1242 non-null   int64         
 1   POSIC                 1242 non-null   int64         
 2   CONCA                 1242 non-null   int64         
 3   DOCUMENTO_REFERENCIA  1242 non-null   int64         
 4   FONDO                 1242 non-null   object        
 5   TIPO_MAT              1242 non-null   object        
 6   TEXTO_BREVE           1242 non-null   object        
 7   CODPROVEEDOR          1242 non-null   int64         
 8   NOMPROVEEDOR          1242 non-null   object        
 9   FECH_CONTAB           1242 non-null   datetime64[ns]
 10  MON                   1242 non-null   object        
 11  INDICADOR             1242 non-null   object        
 12  EA_EM                 1242 non-null   float64       
 13  FACTURADO             0

## Cargo PAP 

In [54]:
PAP = load_df_by_name([r'D:\Scripts1\Code\ActPEA\archvis\PAP'],Today_D_M) #Solo cargar el primero de la fecha es aqui XD 
PAP = PAP[0]
PAP_A = get_recent_df(r'D:\Scripts1\Code\ActPEA\archvis\PAP\Administrativo','Hoja2')# PAP de Admin

PAP = pd.concat([PAP,PAP_A])
#PAP = pd.read_excel(r'D:\Scripts1\Code\ActPEA\archvis\PAP\PAP 19.04.xlsx',sheet_name = 'Hoja2') 

D:\Scripts1\Code\ActPEA\archvis\PAP\Administrativo\PAP_A_30.07.2024_2.xlsx


# Preprocesing
    Paso01

## Preprosecinf of PAP & PREP

In [48]:
PAP_f = PAP[['OC Posición','N° Sol','Estado','Nombre responsable','Id.SIte',
             'SIte','# Días','F.Creación','Gerencia','F.Modifica']]
##Filtro solo del norte
PAP_f = PAP_f.dropna(subset=['Id.SIte'])
PAP_f = PAP_f[PAP_f['# Días'] < 500]

#PAP_f_N = PAP_f[PAP_f['Id.SIte'].str.startswith(('L','T','SAD','CL','CAC'))].copy()
PAP_f_N = PAP_f.rename(columns={'N° Sol': 'PAP', #Rename
                               'SIte': 'SITE',
                               'Estado': 'ESTADO_PAP',
                                '# Días': 'ANTIGUAMIENTO_PAP',
                                'Id.SIte': 'ID Site',
                             'Nombre responsable': 'RESPONSABLE_PAP' })
PAP_S = Ac.split_ocs(PAP_f_N) # Spliteo OCs
PAP_S['OC Posición'] = PAP_S['OC Posición'].astype(str)
######################################################### PREP
Prepa_CI = Prepa_N[Prepa_N.SUB_DIRECCION == 'CONS E IMP']
Prepa_PR = Ac.convert_columns_to_str(Prepa_CI.copy(),['DOC_COMPRAS', 'POSIC','CONCA'])
Prepa_PR.loc[:, "OC Posición"] = Prepa_PR.loc[:, "DOC_COMPRAS"].str.cat(Prepa_PR.loc[:, "POSIC"], sep= ":")
Monto_In = Prepa_PR["PEND_FACT_SOLES"].sum() #Mont of USD 
Prepa_PR = Prepa_PR.dropna(axis=1,how='all')


# Paso 02(Seudo-merge), act columnas 
>Añado columnas

In [49]:
PRE_all_act = update_from_PAP(Prepa_PR,PAP_S)
#PRE_all_act = PRE_all_act[PRE_all_act['STATUS FINAL'] == 'PENDIENTE']
display(PRE_all_act.ESTADO_PAP.value_counts(dropna=False))
### Mapeo Compuesto 
#Modifico el valor par adetemrina condicion
PRE_all_act_maped = map_estados(PRE_all_act)
a = PRE_all_act_maped.ESTADO.value_counts().reset_index()
a.to_csv(f'D:/Prepa_N/Recuento_estados-{Today_D_M}.csv',index_label=False)
a

ValueError: unconverted data remains when parsing with format "%d/%m/%Y %I:%M:%S %p": "AM", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

# 2 FILL missing values by Tablas SAP

## Cargo Tabla SAP ya unida

In [ ]:
df_combinado = pd.read_csv(r'D:\Scripts1\Code\ActPEA\CODE\Temps\SAPCombi.csv')
df_combinado = Ac.convert_columns_to_str(df_combinado,['OC','Pos'])
df_combinado.loc[:, "CONCATENADO"] = df_combinado.loc[:, "OC"].str.cat(df_combinado.loc[:, "Pos"], sep= "")
SAP_4_use = df_combinado[['CONCATENADO','PEP Desc','Fecha OC']]
map_nameSITE = {'TJ5125-SANTIAGO DE CHUCO' : 'TJ5125-SANTIAGO_DE_CHUCO'}
SAP_4_use.loc[:,'PEP Desc'] = SAP_4_use['PEP Desc'].replace(map_nameSITE)
SAP_4_use = SAP_4_use.drop_duplicates()
PRE_all_act_maped.rename(columns={'CONCA':'CONCATENADO','ID Site':'ID_SITIO'},inplace=True)
PRE_all_act_maped = Ac.addSite(PRE_all_act_maped,SAP_4_use) # Añado sitio

In [ ]:
find_ocs_sinSITE(PRE_all_act_maped)

# Agrego antiguamiento para los proyectos sin registrar
    Calculo para todas las filas la diferencia pero solo asigno el valor al antigueamiento en las filas que cumplan la condicion

In [ ]:
PRE_all_act_maped['Time_diff'] = datetime.date.today() - PRE_all_act_maped['FECH_CONTAB'].dt.date
PRE_all_act_maped['Time_diff'] = PRE_all_act_maped['Time_diff'].astype(str)
PRE_all_act_maped['ANTIGUAMIENTO_PAP'] = PRE_all_act_maped['ANTIGUAMIENTO_PAP'].astype(str)
PRE_all_act_maped['Time_diff'] = PRE_all_act_maped['Time_diff'].str.split(' ').str[0]
PRE_all_act_maped.loc[PRE_all_act_maped['ESTADO'] == 'Sin Registrar', 'ANTIGUAMIENTO_PAP'] = PRE_all_act_maped['Time_diff']
## Drop usseless columns


# Capa de plata

In [ ]:
### REcupero lista de columnas a usar
with open(r"D:\Scripts1\Code\ActPEA\CODE\Temps\mi_lista.pickle", "rb") as archivo:
    lista_recuperada = pickle.load(archivo)
columns_E = lista_recuperada
Prepa_silver = PRE_all_act_maped[columns_E]
#Prepa_silver.ESTADO_CENFILE.replace(0,'No Aplica',inplace=True)
Prepa_silver.to_csv(fr'D:\Prepa_N\Prepas\Prepa_N{Today_str}.csv',index=False) # guardo historico de el de plata
print(f'Prepa_N al {Today_str} Guardado')

## Añado linea Al TS 


In [ ]:
Prepa_silver.columns = Prepa_silver.columns.str.strip().str.replace(' ','')

In [13]:
Prepa_silver.info()

<class 'pandas.core.frame.DataFrame'>
Index: 410 entries, 6 to 723
Data columns (total 31 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   DOC_COMPRAS           410 non-null    object        
 1   POSIC                 410 non-null    object        
 2   CONCATENADO           410 non-null    object        
 3   DOCUMENTO_REFERENCIA  410 non-null    int64         
 4   FONDO                 410 non-null    object        
 5   TIPO_MAT              410 non-null    object        
 6   TEXTO_BREVE           410 non-null    object        
 7   CODPROVEEDOR          410 non-null    int64         
 8   NOMPROVEEDOR          410 non-null    object        
 9   FECH_CONTAB           410 non-null    datetime64[ns]
 10  INDICADOR             410 non-null    object        
 11  EA_EM                 410 non-null    float64       
 12  PEND_FACT             410 non-null    float64       
 13  PEND_FACT_SOLES       410

In [14]:
## Como normalizo el nombre de columna ? 

In [15]:
# Nombre del archivo donde se almacenará la fecha
filename = 'last_run_2.json'
# Carga la última fecha de ejecución
last_run_date = Ac.load_last_run_date(filename)
# Comprueba si la celda ya se ha ejecutado hoy
if last_run_date != datetime.datetime.now().date():
    # Tu código aquí
    print('Se añade la linea de: ',Today_str,'al TS')
    ### Tener cuidado que solo se ejecuta una vez al dia
    new_line_TS(Prepa_silver,r'\\LIMBIPBICOV01.claro.pe\Red Región Norte\DashBoardNacional\Prepa_TS1.csv',Today_str,
                                                           ['TIME','NOMPROVEEDOR','RESPONSABLE2','MES-COMPROMISO']) 
    # Guarda la fecha de hoy como la última fecha de ejecución
    Ac.save_last_run_date(filename)
else:
    print("El código ya se ha ejecutado hoy.")

Se añade la linea de:  30-07-2024 al TS


C:\Users\C26764\AppData\Local\Temp\ipykernel_4072\3930541988.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TIME'] = Fecha


# Capa GOLD
--------------------------------------
    -Calculo la instancia de tiempo, con esa etiqueta extraigo los archivos correspondientes, para mergearlos luego agruparlos y luego calcular la diferencia en base a ese temporal.
    (provicional hasta que tenga DB de Nacionales)

In [16]:
# Calcula la fecha de la semana pasada
Today_date = datetime.datetime.strptime(Today_str, '%d-%m-%Y')
semana_pasada = Today_date  - datetime.timedelta(weeks=1)
# Conviértela a cadena de texto
semana_pasada_str = semana_pasada.strftime('%d-%m-%Y')
date_last_week=  semana_pasada_str[:5].replace('-','.')
print('La fecha con la se va a comparar es: ',date_last_week)

La fecha con la se va a comparar es:  23.07


### Creo el Temp-past


#### @TODO Cambiar la funcion que busca el mas antiguo a una mas robusta que busque el mas antiguo pero mas cercano a la fecha que le voy a pasar 
#### 

In [20]:
df = load_df_by_name([r'\\LIMBIPBICOV01.claro.pe\Red Región Norte\Resultados_prepa',r'D:\Scripts1\Code\ActPEA\archvis\PAP'],date_last_week)
#EL INDEX1 ES EL PAP, EL 0 ES EL PREPA
df_silver_past = Merge_data(df[1],df[0])
print('Se creo el df del pasado')

Error: Worksheet named 'Hoja2' not found. Trying with 'Sheet1'
-4.656612873077393e-10
Se encuentra la columan 'STATUSFINAL' y se dropeand las filas de mas
Se creo el df del pasado


In [21]:
Last_silver = get_recent_df(r'D:\Prepa_N\Prepas',extension=".csv") # en caso quiera cargar el df que ya hize para compararlo

D:\Prepa_N\Prepas\Prepa_N30-07-2024.csv


In [22]:
df_gold = calc_diff(Last_silver,df_silver_past,['RESPONSABLE2','NOMPROVEEDOR']) ##Calculo diff con esta agrupacion
df_gold['DIFERENCIA EN SOLES'] = df_gold['DIFERENCIA EN SOLES'].transform(lambda x: x / df_gold['DIFERENCIA EN SOLES'].value_counts()[x] if pd.notnull(x) else x) #Spliteo montos en filas
df_gold_Formated = formated2bi(df_gold) #Formateo
display(df_gold_Formated.info())
df_gold_Formated.to_csv(r'\\LIMBIPBICOV01.claro.pe\Red Región Norte\DashBoardNacional\PrepasivoNacional.csv',index= False)
print('DF GOLD Creado y guardado')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 410 entries, 0 to 409
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Orden De Compra       410 non-null    int64  
 1   Posición              410 non-null    int64  
 2   Documento Referencia  410 non-null    int64  
 3   Descripción           410 non-null    object 
 4   Codproveedor          410 non-null    int64  
 5   Contratista           410 non-null    object 
 6   Fecha Contabilidad    410 non-null    object 
 7   Pendiente En Soles    410 non-null    float64
 8   Demora                410 non-null    int64  
 9   Antiguamiento         410 non-null    object 
 10  Responsable2          410 non-null    object 
 11  Ce Gestor             410 non-null    object 
 12  Fecha Reporte         410 non-null    object 
 13  Fechas Compromiso     410 non-null    object 
 14  Mes De Compromiso     410 non-null    object 
 15  Id Sitio              4

None

DF GOLD Creado y guardado


# PAra crear un modelo estrella:
-Del GOLD
--------------------------------------

### Cargo la data necesaria para ello

In [23]:
Prepa_N =  pd.read_csv(r'\\LIMBIPBICOV01.claro.pe\Red Región Norte\DashBoardNacional\PrepasivoNacional.csv') 
Prepa_N.Estado = Prepa_N.Estado.str.title()
Estado_dim = pd.read_csv(r'D:\Scripts1\Code\ActPEA\CODE\Temps\estados.txt',delim_whitespace=True)  # Cargo los estados y su orden 
Estado_dim.drop(columns=['Índice'],inplace=True)
Estado_dim.Estado = Estado_dim.Estado.str.replace('_',' ')
Estado_dim.Estado = Estado_dim.Estado.str.title()
Estado_dim.Estado = Estado_dim.Estado.str.strip()
hechos = ['Orden De Compra','Posición','Documento Referencia','Descripción',
          'Fecha Contabilidad','Pendiente En Soles','Demora','Antiguamiento','Ce Gestor'
          ,'Id Sitio','Antiguamiento Pap','N° Pap','Site','Diferencia En Soles']
Prepa_N['Mes De Compromiso'] = Prepa_N['Mes De Compromiso'].replace('Setiembre','Septiembre')

In [24]:
Estado_dim.Estado = Estado_dim.Estado.replace('Julio Arceniega','Julio Arciniega')

In [25]:
Contrata_dim = Prepa_N[['Codproveedor','Contratista']].drop_duplicates().reset_index(drop=True)
Responsable_dim = Prepa_N['Responsable2'].drop_duplicates().reset_index(drop=True)
Mes_dim = Prepa_N['Mes De Compromiso'].drop_duplicates().reset_index(drop=True)

#Agrego index a cada dimension
Contrata_dim = Contrata_dim.reset_index().rename(columns={'index': 'Contrata_id'})#Este es perma
Responsable_dim = Responsable_dim.reset_index().rename(columns={'index': 'Responsable_id'})#Este es perma
Mes_dim = Mes_dim.reset_index().rename(columns={'index': 'Mes_id'})# Este aun falta conca
Estado_dim = Estado_dim.reset_index().rename(columns={'index': 'Estado_id'})# Este es perma
Estado_dim.Estado = Estado_dim.Estado.str.strip()

In [26]:
Prepa_N.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 410 entries, 0 to 409
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Orden De Compra       410 non-null    int64  
 1   Posición              410 non-null    int64  
 2   Documento Referencia  410 non-null    int64  
 3   Descripción           410 non-null    object 
 4   Codproveedor          410 non-null    int64  
 5   Contratista           410 non-null    object 
 6   Fecha Contabilidad    410 non-null    object 
 7   Pendiente En Soles    410 non-null    float64
 8   Demora                410 non-null    int64  
 9   Antiguamiento         410 non-null    object 
 10  Responsable2          410 non-null    object 
 11  Ce Gestor             410 non-null    object 
 12  Fecha Reporte         410 non-null    object 
 13  Fechas Compromiso     410 non-null    object 
 14  Mes De Compromiso     410 non-null    object 
 15  Id Sitio              4

In [27]:
## Dimension estado 
Estado_dim1 = Prepa_N['Estado'].drop_duplicates().reset_index(drop=True)
Estado_dim1 = Estado_dim1.reset_index().rename(columns={'index': 'Estado_id'})
Estado_dim1_T = pd.concat([Estado_dim,Estado_dim1])
Estado_dim1_T = Estado_dim1_T.drop_duplicates(subset='Estado')
Estado_dim1_T['Estado_id'] = range(0,len(Estado_dim1_T))

## Del TIME SERIES 
> Normalizar todo, concat, unique, merge y export

In [28]:
TS = pd.read_csv(r'\\LIMBIPBICOV01.claro.pe\Red Región Norte\DashBoardNacional\Prepa_TS1.csv')
TS.rename(columns={'NOMPROVEEDOR':'Contratista',
                  'RESPONSABLE2':'Responsable2',
                  'MES-COMPROMISO':'Mes De Compromiso'},inplace=True)
TS.Contratista = TS.Contratista.str.title()
hechos2 = ['PEND_FACT_SOLES']
TS['Mes De Compromiso'] = TS['Mes De Compromiso'].replace('Setiembre','Septiembre')

In [29]:
## Dimension Contrat
Contrata_dim1 = TS['Contratista'].drop_duplicates().reset_index(drop=True)
Contrata_dim1 = Contrata_dim1.reset_index().rename(columns={'index': 'Contrata_id'})
Contrata_dim_T = pd.concat([Contrata_dim1,Contrata_dim])
Contrata_dim_T = Contrata_dim_T['Contratista'].drop_duplicates().reset_index(drop=True)
Contrata_dim_T = Contrata_dim_T.reset_index().rename(columns={'index': 'Contrata_id'})

In [30]:
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')


'es_ES.UTF-8'

In [31]:
## Dimension MES
Mes_dim1 = TS['Mes De Compromiso'].drop_duplicates().reset_index(drop=True)
Mes_dim1 = Mes_dim1.reset_index().rename(columns={'index': 'Mes_id'})
Mes_dim_T = pd.concat([Mes_dim1,Mes_dim])
Mes_dim_T = Mes_dim_T['Mes De Compromiso'].drop_duplicates().reset_index(drop=True)
Mes_dim_T = Mes_dim_T.reset_index().rename(columns={'index': 'Mes_id'})
Mes_dim_T.loc[:,'Mes De Compromiso'] = Mes_dim_T.loc[:,'Mes De Compromiso'].replace('Setiembre','Septiembre')
Mes_dim_T = Mes_dim_T[~Mes_dim_T['Mes De Compromiso'].isin(['Nuevas EA','Pendiente'])].copy()

Mes_dim_T.loc[:,'Mes De Compromiso'] = Mes_dim_T['Mes De Compromiso'].str.lower()
Mes_dim_T.loc[:,'Numero Mes'] = Mes_dim_T['Mes De Compromiso'].apply(lambda x: list(calendar.month_name).index(x))
Mes_dim_T.loc[:,'Mes De Compromiso'] = Mes_dim_T['Mes De Compromiso'].str.title()

In [32]:
Mes_dim_T

,Mes_id,Mes De Compromiso,Numero Mes
0,0,Marzo,3
1,1,Febrero,2
2,2,Abril,4
3,3,Mayo,5
4,4,Enero,1
5,5,Agosto,8
6,6,Julio,7
7,7,Junio,6
10,10,Octubre,10
11,11,Septiembre,9


### Creo tabla calendar
 - 1. Agarrro las columnas de fehcas de las 2 tablas, las concateno, limpio duplicados, ordeno.
   2. Maximo y minimo en variables
   3. Con eso creo mi tabla calendar y sus demas columnas
   4. merge para asignar el id de cada fecha

In [33]:
dias_abreviados = ['Lun', 'Mar', 'Mie', 'Jue', 'Vie', 'Sab', 'Dom']

# Función para formatear la fecha
def formatear_fecha(fecha):
    dia_abreviado = dias_abreviados[fecha.weekday()]
    return f"{dia_abreviado},{fecha.strftime('%d-%m-%y')}"

In [34]:
Time = pd.concat([Prepa_N['Fechas Compromiso'],TS['TIME']])
Time_df = Time.to_frame()
Time_df.columns = ['Time']
Time_df = Time_df.drop_duplicates()
locale.setlocale(locale.LC_ALL, '')
# Convertir la columna 'Time' a datetime, forzando errores a NaT
Time_df['Time'] = pd.to_datetime(Time_df['Time'], errors='coerce')
# Eliminar filas con NaT en la columna 'Time'
Time_df = Time_df.dropna(subset=['Time'])
# Ordenar el DataFrame por la columna 'Time'
Time_df = Time_df.sort_values(by='Time', ascending=True).reset_index(drop=True)
# Define las fechas de inicio y fin
start_date = Time_df['Time'].min()
end_date = Time_df['Time'].max()
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')
# Crea un rango de fechas
date_range = pd.date_range(start=start_date, end=end_date)

# Crea un DataFrame a partir del rango de fechas
Calendar_df = pd.DataFrame(date_range, columns=['Date'])

# Extrae el año, mes, día, semana, día de la semana, trimestre de la fecha
Calendar_df['Year'] = Calendar_df['Date'].dt.year
Calendar_df['Month_id'] = Calendar_df['Date'].dt.month
Calendar_df['Day'] = Calendar_df['Date'].dt.day
Calendar_df.rename(columns={'Date':'TIME'},inplace=True)



locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')  # Ajusta según tu sistema operativo
Calendar_df['Date-format'] = Calendar_df['TIME'].apply(formatear_fecha)

In [35]:
Calendar_df = Calendar_df.reset_index().rename(columns={'index': 'Date_id'})# Este es perma

In [36]:
Calendar_df

,Date_id,TIME,Year,Month_id,Day,Date-format
0,0,2024-03-01,2024,3,1,"Vie,01-03-24"
1,1,2024-03-02,2024,3,2,"Sab,02-03-24"
2,2,2024-03-03,2024,3,3,"Dom,03-03-24"
3,3,2024-03-04,2024,3,4,"Lun,04-03-24"
4,4,2024-03-05,2024,3,5,"Mar,05-03-24"
...,...,...,...,...,...,...
300,300,2024-12-26,2024,12,26,"Jue,26-12-24"
301,301,2024-12-27,2024,12,27,"Vie,27-12-24"
302,302,2024-12-28,2024,12,28,"Sab,28-12-24"
303,303,2024-12-29,2024,12,29,"Dom,29-12-24"


## Relaciono tablas y Creo tabla de Hechos 

In [37]:
# Crea una tabla de hechos
hechos_df = Prepa_N.copy()
hechos_df = pd.merge(hechos_df, Contrata_dim_T, on='Contratista',how='left')
hechos_df = pd.merge(hechos_df, Responsable_dim, on='Responsable2',how='left')
hechos_df = pd.merge(hechos_df, Mes_dim_T, on='Mes De Compromiso',how='left')
hechos_df = pd.merge(hechos_df, Estado_dim1_T, on='Estado',how='left')

fact_2bi = hechos_df[['Contrata_id', 'Responsable_id','Mes_id','Estado_id'] + hechos]

In [38]:
# Crea una tabla de hechos
hechos2_df = TS.copy()
hechos2_df['TIME'] = pd.to_datetime(hechos2_df['TIME'])
hechos2_df = pd.merge(hechos2_df, Contrata_dim_T, on='Contratista',how='left')
hechos2_df = pd.merge(hechos2_df, Responsable_dim, on='Responsable2',how='left')
hechos2_df = pd.merge(hechos2_df, Mes_dim_T, on='Mes De Compromiso',how='left')
hechos2_df = pd.merge(hechos2_df, Calendar_df[['Date_id','TIME']], on='TIME',how='left')

#hechos2_df = pd.merge(hechos2_df, Contrata_dim, on='Codproveedor',how='left')


facTS_2bi = hechos2_df[['Responsable_id','Mes_id','Contrata_id','Date_id'] + hechos2]

In [39]:
#Estaticas
Contrata_dim_T.to_csv(r'\\LIMBIPBICOV01.claro.pe\Red Región Norte\DashBoardNacional\DATA\Contrata_dim.csv',index=False)
Responsable_dim.to_csv(r'\\LIMBIPBICOV01.claro.pe\Red Región Norte\DashBoardNacional\DATA\Responsable_dim.csv',index=False)
Estado_dim1_T.to_csv(r'\\LIMBIPBICOV01.claro.pe\Red Región Norte\DashBoardNacional\DATA\Estado_dim.csv',index=False)

In [40]:
#Casi dinamicas
Mes_dim_T.to_csv(r'\\LIMBIPBICOV01.claro.pe\Red Región Norte\DashBoardNacional\DATA\Mes_dim.csv',index=False)
Calendar_df.to_csv(r'\\LIMBIPBICOV01.claro.pe\Red Región Norte\DashBoardNacional\DATA\Calendar_table.csv',index=False)


In [41]:
#Dinamicas
facTS_2bi.to_csv( r'\\LIMBIPBICOV01.claro.pe\Red Región Norte\DashBoardNacional\DATA\TS_N_fact.csv',index=False)
fact_2bi.to_csv(r'\\LIMBIPBICOV01.claro.pe\Red Región Norte\DashBoardNacional\DATA\hechos_df.csv',index=False)


In [42]:
Calendar_df

,Date_id,TIME,Year,Month_id,Day,Date-format
0,0,2024-03-01,2024,3,1,"Vie,01-03-24"
1,1,2024-03-02,2024,3,2,"Sab,02-03-24"
2,2,2024-03-03,2024,3,3,"Dom,03-03-24"
3,3,2024-03-04,2024,3,4,"Lun,04-03-24"
4,4,2024-03-05,2024,3,5,"Mar,05-03-24"
...,...,...,...,...,...,...
300,300,2024-12-26,2024,12,26,"Jue,26-12-24"
301,301,2024-12-27,2024,12,27,"Vie,27-12-24"
302,302,2024-12-28,2024,12,28,"Sab,28-12-24"
303,303,2024-12-29,2024,12,29,"Dom,29-12-24"
